In [ ]:
import zipfile, os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# 경로 설정
aug_zip_dir = '/content/drive/MyDrive/ MyDrive BrainTumorDataset '
raw_dataset_dir = '/content/BrainTumorRaw/multi_class_dataset'

# ZIP 압축 해제
zip_map = {
    'Glioma.zip': 'glioma',
    'Meningioma.zip': 'meningioma',
    'Pituitary.zip': 'pituitary',
    'Healthy.zip': 'healthy'
}
for zip_file, class_name in zip_map.items():
    zip_path = os.path.join(aug_zip_dir, zip_file)
    target_dir = os.path.join(raw_dataset_dir, class_name)
    os.makedirs(target_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)

print("✅ 증강 이미지 병합 완료:", os.listdir(raw_dataset_dir))

# 데이터 로딩 및 증강
img_size = (224, 224)
batch_size = 64

train_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

train_data = train_gen.flow_from_directory(
    raw_dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = val_gen.flow_from_directory(
    raw_dataset_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

num_classes = len(train_data.class_indices)

# ✅ 콜백 설정
earlystop = EarlyStopping(
    monitor='val_loss',
    patience=7,  # 넉넉히
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'model.keras',
    monitor='val_accuracy',
    save_best_only=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=4,  # 4번 연속 개선 없을 때만 감소
    min_lr=1e-7,  # 너무 작아지는 것 방지
    verbose=1
)

# MLP 모델 정의
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=img_size + (3,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🔵 [MLP 모델 학습 시작]")
history_mlp = mlp_model.fit(
    train_data,
    epochs=40,
    validation_data=val_data,
    callbacks=[earlystop, checkpoint, reduce_lr]
)

# DNN 모델 정의
dnn_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=img_size + (3,)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

dnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🟣 [DNN 모델 학습 시작]")
history_dnn = dnn_model.fit(
    train_data,
    epochs=40,
    validation_data=val_data,
    callbacks=[earlystop, checkpoint, reduce_lr]
)

# 정확도 시각화
def plot_history(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    plt.plot(acc, label='Train Accuracy')
    plt.plot(val_acc, label='Val Accuracy')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_history(history_mlp, "MLP 정확도")
plot_history(history_dnn, "DNN 정확도")

print("✅ MLP 최종 검증 정확도:", mlp_model.evaluate(val_data, verbose=0)[1])
print("✅ DNN 최종 검증 정확도:", dnn_model.evaluate(val_data, verbose=0)[1])


✅ 증강 이미지 병합 완료: ['glioma', 'healthy', 'pituitary', 'meningioma']
Found 34167 images belonging to 4 classes.
Found 14640 images belonging to 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🔵 [MLP 모델 학습 시작]


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
534/534 ━━━━━━━━━━━━━━━━━━━━ 387s 715ms/step - accuracy: 0.4102 - loss: 4.9771 - val_accuracy: 0.3961 - val_loss: 1.2804 - learning_rate: 0.0010
Epoch 2/40
534/534 ━━━━━━━━━━━━━━━━━━━━ 372s 696ms/step - accuracy: 0.4920 - loss: 1.2096 - val_accuracy: 0.3805 - val_loss: 1.3632 - learning_rate: 0.0010
Epoch 3/40
534/534 ━━━━━━━━━━━━━━━━━━━━ 374s 701ms/step - accuracy: 0.5052 - loss: 1.1613 - val_accuracy: 0.3542 - val_loss: 1.3810 - learning_rate: 0.0010
Epoch 4/40
534/534 ━━━━━━━━━━━━━━━━━━━━ 374s 701ms/step - accuracy: 0.5147 - loss: 1.1422 - val_accuracy: 0.4171 - val_loss: 1.2360 - learning_rate: 0.0010
Epoch 5/40
534/534 ━━━━━━━━━━━━━━━━━━━━ 370s 692ms/step - accuracy: 0.5131 - loss: 1.1502 - val_accuracy: 0.3932 - val_loss: 1.3238 - learning_rate: 0.0010
Epoch 6/40
534/534 ━━━━━━━━━━━━━━━━━━━━ 372s 696ms/step - accuracy: 0.5278 - loss: 1.1306 - val_accuracy: 0.4001 - val_loss: 1.3071 - learning_rate: 0.0010
Epoch 7/40
534/534 ━━━━━━━━━━━━━━━━━━━━ 373s 699ms/step - accura